# PIPELINE

In [1]:
import kfp

from typing import NamedTuple
import kfp.components as comp
from kfp import compiler, dsl
from kfp import dsl
from kfp.components import InputPath, OutputPath
from kubernetes.client.models import V1EnvVar

In [2]:
def GenerateData(heat_consumption_path: OutputPath(str), heat_supply_path: OutputPath(str), weather_info_path : OutputPath(str), time_info_path: OutputPath(str)):
    
    from datetime import datetime
    import maya
    import random
    import numpy as np
    import requests
    import json
    
    def SimulateTSValue(td = maya.now(), trend_coef = 0.5, trend_sd = 0.025, seed_offset = 0):
        # Random Trend component around 0.5
        # Definition: 0.5 + uniform random number between -0.025 and 0.025
        trend_ts = trend_coef + random.uniform(-trend_sd,trend_sd)
    
        # Random Weekly component
        # Definition Component Based on the day of the week fixed numver + uniform error
        list_comp_days = [0.2,0.23,0.26,0.24,0.21,0.18,0.18, 0.18]
        sd_days = [0.01,0.02,0.01,0.02,0.01,0.01,0.005, 0.005]
    
        day_of_week = td.weekday
        component_weekly = list_comp_days[day_of_week] + random.uniform(-sd_days[day_of_week], sd_days[day_of_week])
    
        # Random Hourly Component
        # Shape based on the day of the simulation
        random.seed(td.day + seed_offset)
        index_shape = random.randint(1,3)
        shape_name = "Shape_" + str(index_shape)
        shapes_dict = {
            "Shape_1":[-0.1       , -0.16165509, -0.19580765, -0.20749836, -0.2017679 ,-0.18365698, -0.15820627, -0.13045646, -0.10544825, -0.08790597,
               -0.07969859, -0.08118035, -0.0926916 , -0.11431926, -0.14248568,-0.17085613, -0.19302855, -0.20271256, -0.19646934, -0.17387802,
               -0.13466275, -0.07860226, -0.0084324 ,  0.06870108,  0.14529271,0.21385705,  0.27034905,  0.31803569,  0.36111925,  0.40380147,
                0.44921795,  0.49721282,  0.54699702,  0.59778148,  0.64873965,0.69887417,  0.74713902,  0.79248819,  0.83404121,  0.87204349,
                0.90720933,  0.94025454,  0.9719917 ,  1.00424743,  1.03945362,1.08005027,  1.12821682,  1.18164127,  1.23422369,  1.27974554,
                1.3121125 ,  1.32919786,  1.33357925,  1.3281073 ,  1.31564286,1.29981316,  1.28552937,  1.27782515,  1.28172227,  1.29863822,
                1.32133515,  1.34130265,  1.35003053,  1.34175905,  1.32038759,1.29193471,  1.26241895,  1.23771333,  1.22293425,  1.22295303,
                1.24264089,  1.28498662,  1.33826295,  1.38381853,  1.40296166,1.37869494,  1.31479044,  1.22894771,  1.13912333,  1.06269959,
                1.00510714,  0.96048399,  0.92252581,  0.8849787 ,  0.84365402,0.79710807,  0.744087  ,  0.68334896,  0.61496013,  0.54145249,
                0.46563119,  0.39030138,  0.31826821,  0.25233682,  0.19531237,0.15],
            "Shape_2":[ 0., -0.04103375, -0.08817198, -0.13946464, -0.1929617 ,-0.24671311, -0.29876883, -0.34717883, -0.38999305, -0.42547586,
               -0.45382685, -0.47627221, -0.49404756, -0.50822958, -0.51759701,-0.51919966, -0.51004514, -0.48723606, -0.45030097, -0.40133591,
               -0.3425603 , -0.27619642, -0.20462236, -0.1304485 , -0.05630419,0.01519295,  0.08343867,  0.15210782,  0.2254226 ,  0.30760414,
                0.4007446 ,  0.50036352,  0.60071601,  0.69605718,  0.78173214,0.85805425,  0.92675155,  0.98955223,  1.04791236,  1.10143735,
                1.14896196,  1.18931843,  1.22156961,  1.24719603,  1.26912132,1.29028841,  1.31348881,  1.3389036 ,  1.36451233,  1.38822559,
                1.40799846,  1.42320642,  1.43490909,  1.44426382,  1.45241457,1.45950627,  1.46401005,  1.46423742,  1.45850549,  1.44684352,
                1.43339228,  1.42289702,  1.42010287,  1.42783964,  1.44221093,1.45784466,  1.46936874,  1.4722723 ,  1.46652136,  1.45353202,
                1.4347211 ,  1.41136867,  1.38368551,  1.35137928,  1.31415472,1.27179803,  1.22509424,  1.17549813,  1.12447684,  1.07347197,
                1.02339291,  0.97464625,  0.92761887,  0.88266371,  0.83874269,0.79296899,  0.74232789,  0.68382295,  0.6164467 ,  0.54294106,
                0.46646335,  0.3901709 ,  0.31722102,  0.25077103,  0.19397825,0.15],
            "Shape_3":[ 0.1,  0.04368145, -0.00611102, -0.05146214, -0.09445662,-0.13717918, -0.18171454, -0.23014741, -0.28456252, -0.34620278,
               -0.40871244, -0.46170489, -0.49475657, -0.49836437, -0.47633539,-0.44249094, -0.41089688, -0.39514781, -0.39680727, -0.40470557,
               -0.40706122, -0.39222805, -0.3558937 , -0.30468233, -0.2461103 ,-0.18767166, -0.13303198, -0.07772009, -0.01622396,  0.05696667,
                0.14399016,  0.23657478,  0.32444615,  0.39732988,  0.44853585,0.48771076,  0.52915333,  0.5871626 ,  0.67308554,  0.77819026,
                0.88538361,  0.97754486,  1.03916537,  1.07163453,  1.08642819,1.09515719,  1.10896923,  1.13102895,  1.15776826,  1.18540828,
                1.21023488,  1.23060292,  1.24732041,  1.26133773,  1.27357663,1.2828239 ,  1.28428948,  1.2728421 ,  1.24336168,  1.19413061,
                1.13160193,  1.06342997,  0.99726893,  0.93901253,  0.888372  ,0.84370217,  0.80335787,  0.7663065 ,  0.73469977,  0.71172084,
                0.70055338,  0.703921  ,  0.72095085,  0.74907786,  0.78572713,0.82837812,  0.87517695,  0.92471675,  0.97559894,  1.02624828,
                1.07141393,  1.10237207,  1.11026281,  1.08642757,  1.03044917,0.95286434,  0.86496734,  0.77801236,  0.69888604,  0.62624165,
                0.55782034,  0.49136324,  0.42461152,  0.35530631,  0.28118875,0.15]
        }
        comp_hour = shapes_dict[shape_name][td.hour*4 + int(td.minute/15)] + random.uniform(-0.01,0.01)
    
        final_value = trend_ts + component_weekly + comp_hour

        return final_value
    
    # CALCULATE VALUES
    td = maya.now()
    gas_boiler1 = SimulateTSValue(td = td, trend_coef = 0, trend_sd = 0.05, seed_offset = 1994)
    gas_boiler2 = SimulateTSValue(td = td, trend_coef = 1, trend_sd = 0.05, seed_offset = 133)
    
    print(1994)

    b1_consumption = SimulateTSValue(td = td, trend_coef = 3, trend_sd = 0.05, seed_offset= 217)
    b2_consumption = SimulateTSValue(td = td, trend_coef = 3, trend_sd = 0.05, seed_offset= 1511)

    url = "https://api.weatherbit.io/v2.0/current?city=Poznan&country=PL&key=ff58bcc61d0445199d950a6fa6d15489"

    payload={}
    headers = {}

    response = requests.request("GET", url, headers=headers, data=payload)
    data_weather = response.json()

    if "data" not in data_weather.keys():
        url = "https://api.weatherbit.io/v2.0/current?city=Poznan&country=PL&key=f6d8ac05e90c4b48a7382a39918a8eca"
        response = requests.request("GET", url, headers=headers, data=payload)
    try:
        data_weather = response.json()
        temperature = data_weather["data"][0]["temp"]
        solar_rad = data_weather["data"][0]["solar_rad"]
        weather_info = data_weather["data"][0]["weather"]["description"]
    except:
        temperature = 15
        solar_rad = 0
        weather_info = "No Info"

    b1_temperature = temperature + np.round(random.uniform(-1.5, 1.5),1)
    b2_temperature = temperature + np.round(random.uniform(-1.5, 1.5),1)

    b1_solar = solar_rad * random.uniform(0.95, 1.05)
    b2_solar = solar_rad * random.uniform(0.95, 1.05)

    solar_collector = np.mean([b1_solar, b2_solar]) * random.uniform(0.965,0.975)/1000*2
    
    data_dict = {
        "building1": b1_consumption,
        "building2": b2_consumption
    }
    with open(heat_consumption_path, "w") as out_file:
        json.dump(data_dict, out_file)
    
    data_dict = {
        "gas_boiler1": gas_boiler1,
        "gas_boiler2": gas_boiler2,
        "solar_collector1": solar_collector
    }
    
    with open(heat_supply_path, "w") as out_file:
        json.dump(data_dict, out_file)
    
    data_dict = {
        "current_temperature": {
            "building1": b1_temperature,
            "building2": b2_temperature
        } ,
        "sun_radiation": {
            "building1": int(b1_solar),
            "building2": int(b2_solar)
        },
        "weather_info": {"energy_island": weather_info}
    }
    
    with open(weather_info_path, "w") as out_file:
        json.dump(data_dict, out_file)
    
    data_dict = {
        "datetime": td.epoch
    }
    
    with open(time_info_path, "w") as out_file:
        json.dump(data_dict, out_file)

In [3]:
def SendData2Influx(measurement_name,input_data_path:InputPath(str), time_info_path: InputPath(str)):
    
    import json
    import requests
    
    with open(input_data_path) as data_file:
        data_str = json.load(data_file)
    with open(time_info_path) as data_file:
        time_str = json.load(data_file)
    
    def GetDictPost(measurement_name, value, asset_name, time_pred):
        if measurement_name in ["current_temperature", "sun_radiation"]:
            if measurement_name == "current_temperature":
                field_name = "temperature"
            elif measurement_name == "sun_radiation":
                field_name = "sun_radiation"
            else:
                field_name = "percentage"
                
            data_post = {
                "bucket": "renergetic",
                "measurement": "weather_API",
                "fields":{
                    field_name: value
                },
                "tags":{
                    "domain": "weather",
                    "typeData": "simulated",
                    "direction": "None",
                    "prediction_window": "Oh",
                    "asset_name": asset_name,
                    "time_prediction": time_pred
                }
            }
        elif measurement_name in ["heat_supply", "heat_consumption"]:
            
            if measurement_name == "heat_supply":
                direction_energy = "out"
            elif measurement_name == "heat_consumption":
                direction_energy = "in"
            else:
                direction_energy = "None"
            
            data_post = {
                "bucket": "renergetic",
                "measurement": "heat_meter",
                "fields":{
                    "energy": value
                },
                "tags":{
                    "domain": "heat",
                    "typeData": "simulated",
                    "direction": direction_energy,
                    "prediction_window": "Oh",
                    "asset_name": asset_name,
                    "time_prediction": time_pred
                }
            }
        else:
            data_post = {}
            
        return data_post


    url =  "http://influx-api-ren-prototype.apps.paas-dev.psnc.pl/api/measurement"

    headers = {
    "Content-Type": "application/json"
    }
    
    msg = "Upload Status:\n"
    
    for key_dict in data_str.keys():
        if measurement_name != "weather_info":
            value = data_str[key_dict]
            asset_name = key_dict
            time_pred = time_str["datetime"]
            
            data_post = GetDictPost(measurement_name, value, asset_name, time_pred)

            response = requests.request("POST", url, headers=headers, data=json.dumps(data_post))
            
            if response.status_code == 200:
                msg = msg + measurement_name + "_" + key_dict + " -> SUCCESS" + "\n"
            else:
                print(data_post)
                msg = msg + measurement_name + "_" + key_dict + " -> FAILED" + "\n"
        else:
            for key_specific in data_str[key_dict].keys():
                measurement_name_spec = key_dict
                value = data_str[key_dict][key_specific]
                asset_name = key_specific
                time_pred = time_str["datetime"]
                
                data_post = GetDictPost(measurement_name_spec, value, asset_name, time_pred)
                
                response = requests.request("POST", url, headers=headers, data=json.dumps(data_post))
                
                if response.status_code == 200:
                    msg = msg + key_dict + "_" + key_specific + " -> SUCCESS" + "\n"
                else:
                    print(measurement_name)
                    print(measurement_name_spec)
                    print(data_post)
                    msg = msg + key_dict + "_" + key_specific + " -> FAILED" + "\n"
                    
    print(msg)  
        

In [4]:
def REN_Forecast_Test_Pipeline():
    env_var = V1EnvVar(name='HOME', value='/tmp')
    generate_data_op = comp.create_component_from_func(
        GenerateData, packages_to_install = ["requests", "numpy", "maya"], output_component_file = "generate_data_op_component.yaml")
    send_simulations_op = comp.create_component_from_func(
        SendData2Influx, packages_to_install = ["requests"], output_component_file = "send_simulations_op_component.yaml")
    
    #TASKS DEFINITION
    
    generate_data_task = (generate_data_op().add_env_variable(env_var)
                        .set_memory_request('500M')
                        .set_memory_limit('1Gi')
                        .set_cpu_request('1')
                        .set_cpu_limit('2'))
    
    send_supply_data_task = (send_simulations_op("heat_supply", 
                                                generate_data_task.outputs['heat_supply'],
                                                generate_data_task.outputs["time_info"]
                                               ).add_env_variable(env_var)
                            .set_memory_request('500M')
                            .set_memory_limit('1Gi')
                            .set_cpu_request('1')
                            .set_cpu_limit('2'))
    
    send_consumption_data_task = (send_simulations_op("heat_consumption", 
                                                generate_data_task.outputs["heat_consumption"],
                                                generate_data_task.outputs["time_info"]
                                               ).add_env_variable(env_var)
                                .set_memory_request('500M')
                                .set_memory_limit('1Gi')
                                .set_cpu_request('1')
                                .set_cpu_limit('2'))
    
    send_weather_data_task = (send_simulations_op("weather_info", 
                                                generate_data_task.outputs["weather_info"],
                                                generate_data_task.outputs["time_info"]
                                               ).add_env_variable(env_var)
                                .set_memory_request('500M')
                                .set_memory_limit('1Gi')
                                .set_cpu_request('1')
                                .set_cpu_limit('2'))


compiler.Compiler().compile(pipeline_func = REN_Forecast_Test_Pipeline, package_path ="Simulate_Data_Pipeline.yaml")

In [5]:
print("Success")

Success
